<a href="https://colab.research.google.com/github/Krazahra/Machine-Learning/blob/main/Tugas_12_RessNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Identitas
Nama: Kinanti Rahayu Az-Zahra

Kelas: TK-45-04

NIM: 1103210052

Dataset: https://www.tensorflow.org/datasets/catalog/mnist

Dokumentasi GPT: https://chatgpt.com/share/5efefc41-d4fb-4cec-91d7-ebf1c6c5147b

In [12]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import mnist

In [13]:
# Load MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Normalize pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0
# Reshape data to include channel dimension (assuming grayscale images)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [14]:
# Define early stopping callback
early_stop = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

In [15]:
# Define ResNet block function
def resnet_block(x, filters, kernel_size=3, stride=1, conv_shortcut=True, name=None):
    bn_axis = 3

    if conv_shortcut:
        shortcut = layers.Conv2D(filters * 4, 1, strides=stride, name=name + '_0_conv')(x)
        shortcut = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_0_bn')(shortcut)
    else:
        shortcut = x

    x = layers.Conv2D(filters, 1, strides=stride, use_bias=False, name=name + '_1_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_1_bn')(x)
    x = layers.Activation('relu', name=name + '_1_relu')(x)

    x = layers.Conv2D(filters, kernel_size, padding='same', use_bias=False, name=name + '_2_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_2_bn')(x)
    x = layers.Activation('relu', name=name + '_2_relu')(x)

    x = layers.Conv2D(filters * 4, 1, use_bias=False, name=name + '_3_conv')(x)
    x = layers.BatchNormalization(axis=bn_axis, epsilon=1.001e-5, name=name + '_3_bn')(x)

    x = layers.Add(name=name + '_add')([shortcut, x])
    x = layers.Activation('relu', name=name + '_out')(x)
    return x

In [16]:
# Define ResNet model function
def resnet(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Conv2D(64, 7, strides=2, padding='same', use_bias=False, name='conv1_conv')(inputs)
    x = layers.BatchNormalization(axis=3, epsilon=1.001e-5, name='conv1_bn')(x)
    x = layers.Activation('relu', name='conv1_relu')(x)
    x = layers.MaxPooling2D(3, strides=2, padding='same', name='pool1')(x)

    x = resnet_block(x, 64, conv_shortcut=True, name='conv2_block1')
    x = resnet_block(x, 64, conv_shortcut=False, name='conv2_block2')

    x = resnet_block(x, 128, stride=2, conv_shortcut=True, name='conv3_block1')
    x = resnet_block(x, 128, conv_shortcut=False, name='conv3_block2')

    x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
    x = layers.Dense(num_classes, activation='softmax', name='predictions')(x)

    model = tf.keras.Model(inputs, x, name='resnet')
    return model

In [17]:
# Create ResNet model
input_shape = (28, 28, 1)
num_classes = 10
model = resnet(input_shape, num_classes)

In [18]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [19]:
# Print model summary
model.summary()

Model: "resnet"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 28, 28, 1)]          0         []                            
                                                                                                  
 conv1_conv (Conv2D)         (None, 14, 14, 64)           3136      ['input_2[0][0]']             
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 14, 14, 64)           256       ['conv1_conv[0][0]']          
 on)                                                                                              
                                                                                                  
 conv1_relu (Activation)     (None, 14, 14, 64)           0         ['conv1_bn[0][0]']       

In [20]:
# Train the model with early stopping
history = model.fit(x_train, y_train, epochs=20, batch_size=128,
                    validation_data=(x_test, y_test), callbacks=[early_stop])

Epoch 1/20
469/469 [==============================] - 21s 21ms/step - loss: 0.1139 - accuracy: 0.9656 - val_loss: 0.7425 - val_accuracy: 0.7703
Epoch 2/20
469/469 [==============================] - 7s 14ms/step - loss: 0.0423 - accuracy: 0.9865 - val_loss: 0.0908 - val_accuracy: 0.9733
Epoch 3/20
469/469 [==============================] - 8s 17ms/step - loss: 0.0320 - accuracy: 0.9897 - val_loss: 0.0707 - val_accuracy: 0.9779
Epoch 4/20
469/469 [==============================] - 7s 15ms/step - loss: 0.0258 - accuracy: 0.9919 - val_loss: 0.0307 - val_accuracy: 0.9907
Epoch 5/20
469/469 [==============================] - 8s 16ms/step - loss: 0.0234 - accuracy: 0.9922 - val_loss: 0.0636 - val_accuracy: 0.9816
Epoch 6/20
469/469 [==============================] - 8s 17ms/step - loss: 0.0158 - accuracy: 0.9951 - val_loss: 0.0783 - val_accuracy: 0.9773
Epoch 7/20
469/469 [==============================] - 7s 16ms/step - loss: 0.0188 - accuracy: 0.9942 - val_loss: 0.0609 - val_accuracy: 0.984

In [21]:
# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

313/313 - 1s - loss: 0.0289 - accuracy: 0.9912 - 1s/epoch - 4ms/step

Test accuracy: 0.9911999702453613
